## Spectral Clustering and Semi-Supervised Learning

#### This notebook contains different values and imports that can be used in this practical session.
#### Please keep the same variable names when provided in your report to make the work of teaching assistants easier. 
#### You can still change the values given or the sizes of the datasets treated if you believe it is usefull to illustrate your point.

#### You can directly edit the markdown boxes to add your comments and answers to the questions

In [ ]:
# Question 1 (imports and advised values):
from sklearn.datasets import make_moons
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

matplotlib.style.use('seaborn-notebook')

n_samples = 200 # You can change these values
noise_level_list = [.05, .1, .2] # You can change these values

### Question 1 : Complete the code in the box below

In [ ]:
#A dictionnary which maps the noise level to the correponding moon dataset (if it has already been generated)
noisy_moons = {} 
for lev in noise_level_list:
    noisy_moons[lev] = make_moons(n_samples, shuffle = False, noise=lev)
    
f, a = plt.subplots(1, len(noise_level_list))
f.subplots_adjust(wspace = 0.5, hspace=0.5)

for i, lev in enumerate(noisy_moons):
    a[i].scatter(noisy_moons[lev][0][:,0], noisy_moons[lev][0][:,1], c = noisy_moons[lev][1])
    a[i].set_title('Noise = %.2f' % lev)

In [ ]:
# Question 2 :
from sklearn.neighbors import kneighbors_graph
n_neighbors_list = [1, 5, 10] # You can change these values

### Question 2 : Complete the code in the box below

In [ ]:
def plot_kneighbors_graph(axes, graph, pts, noise, k):
    axes.scatter(pts[:, 0], pts[:, 1], marker='.')
    xidx, yidx = graph.nonzero()
    for i, j in zip(xidx, yidx):
        if i < j:
            axes.plot([pts[i, 0], pts[j, 0]], [pts[i, 1], pts[j, 1]], linewidth=0.5, c='g')
    axes.set_title('Noise =  %.2f, k = %.1f' % (noise, k),fontsize = 10)

In [ ]:
#Generate the adjacency matrices
kNN_graphs = {}
for k in n_neighbors_list:
    for lev in noise_level_list:
        kNN_graphs[k, lev] = kneighbors_graph(noisy_moons[lev][0], k, mode='distance', p=2)

f, a = plt.subplots(len(n_neighbors_list), len(noise_level_list))
f.subplots_adjust(wspace = 0.5, hspace=0.5)
for i, k in enumerate(n_neighbors_list):
    for j, lev in enumerate(noise_level_list):
        plot_kneighbors_graph(a[i, j], kNN_graphs[k, lev], noisy_moons[lev][0], lev, k)
        #a[i, j].imshow(kNN_graphs[k, lev].todense(), cmap = 'Greys', interpolation="none")
        #a[i, j].set_title('Noise =  %.2f, k = %.1f' % (lev, k),fontsize = 10)

### Question 3 : Optimization problem 

- Given a graph $\mathcal{G} = (E,V)$, let $D$ be its degree matrix and $W$ its weighted ajacency matrix, such that $L := D - W$ is the Laplacian of the graph. In the case when we consider a partition in 2 subsets $A,\bar{A}$, the Normalized Cut problem translates as the following optimization problem:



- $\min_A \{ f^\top L f \} \text{ s.t. } f_i = 
\begin{cases}
    \sqrt{\frac{\text{Vol}(\bar{A})}{\text{Vol}(A)}},& \text{if } v_i\in A\\
    \sqrt{\frac{\text{Vol}(A)}{\text{Vol}(\bar{A})}},& \text{if } v_i\in \bar{A}
\end{cases}, Df \bot \mathbb{1}, f^\top D f = \text{Vol}(V)$


- This problem is then relaxed as $\min_{f\in \mathbb{R}^n} f^\top L f \text{ s.t. } Df \bot \mathbb{1}, f^\top D f = \text{Vol}(V)$


### Question 4 : Complete the code in the box below



In [ ]:
from sklearn.cluster import SpectralClustering

f, a = plt.subplots(len(n_neighbors_list), len(noise_level_list))
f.subplots_adjust(wspace = 0.5, hspace=0.5)
labels = {}
for i, k in enumerate(n_neighbors_list):
    for j, l in enumerate(noise_level_list):
        spectral = SpectralClustering(n_clusters=2, eigen_solver='arpack', affinity="precomputed")
        labels[k, l] = spectral.fit_predict(kneighbors_graph(noisy_moons[l][0], k, mode='distance', p=2).todense())
        a[i, j].scatter(noisy_moons[l][0][:,0], noisy_moons[l][0][:,1], c = labels[k, l])
        a[i, j].set_title('Noise =  %.2f, k = %.1f' % (l, k), fontsize = 10)


It seems that increasing the number improves the robustness of the clustering.
However, since it makes the adjacency matrix denser, it also makes the complexity greater.

### Question 5 : Complete the code in the box below

In [ ]:
#The Frobenius inner product for matrices
def frobenius(A, B):
    return np.sum(np.multiply(A,B))

#Compute the matrix representation C of a given clustering
#ie Cij = 1 iff xi and xj are clustered together and i != j
def matrix_clustering(labels):
    n = len(labels)
    C = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            if (labels[i] == labels[j] and i != j):
                C[i,j] = 1
    return C

def cosine_similarity(C1, C2):
    return frobenius(C1, C2) / (np.linalg.norm(C1, 'fro')*np.linalg.norm(C2, 'fro'))

#Return the average cosine similarity between a reference clustering and bootstrapped clusterings over B runs
def cluster_stability(X, algo, clusters=2, B=10, f=0.8):
    n = X.shape[0]
    ref = algo(X, clusters)
    
    avg_similarity = 0
    
    for i in range(B):
        indices = np.random.choice(n, f * n, replace=False)
        bootstrapped = X[indices, :]
        c = algo(bootstrapped, clusters)
        avg_similarity += cosine_similarity(matrix_clustering(c), matrix_clustering(ref[indices]))
        
    return avg_similarity / B

### Question 6 : Complete the code in the box below

Let us plot the stability as a function of the number of neighbors used for spectral clustering, on noisy and non-noisy versions of the moon dataset.

In [ ]:
noise_free_stability = []
noisy_stability = []
n_neighbors_list = np.arange(1,11,2)
for k in n_neighbors_list:
    noisy_stability.append(
        cluster_stability(noisy_moons[.2][0], 
            (lambda X, clusters: SpectralClustering(n_clusters=clusters, 
                            affinity="precomputed").fit_predict(kneighbors_graph(X, k, mode='distance', p=2).todense())),
            B=50, f=0.8))
    noise_free_stability.append(
        cluster_stability(noisy_moons[.05][0], 
            (lambda X, clusters: SpectralClustering(n_clusters=clusters, 
                            affinity="precomputed").fit_predict(kneighbors_graph(X, k, mode='distance', p=2).todense())), 
            B=50, f=0.8))

plt.plot(n_neighbors_list, noisy_stability, label = "Noisy")
plt.plot(n_neighbors_list, noise_free_stability, label = "Noise-free")
plt.title("Stability of Spectral Clustering as a function of the number of neighbors")
plt.legend()

### Question 7 : Complete the code in the box below

In [ ]:
from sklearn.cluster import AgglomerativeClustering

noise_free_stability = []
noisy_stability = []
n_neighbors_list = np.arange(1,11,2)

for k in n_neighbors_list:
    noisy_stability.append(
        cluster_stability(noisy_moons[.2][0], 
            (lambda X, clusters: AgglomerativeClustering(n_clusters=2, affinity='euclidean', 
            connectivity=kneighbors_graph(X, k, mode='distance', p=2).todense(), linkage='ward').fit_predict(X)),B=20, f=0.8))
    noise_free_stability.append(
        cluster_stability(noisy_moons[.05][0], 
            (lambda X, clusters: AgglomerativeClustering(n_clusters=2, affinity='euclidean', 
            connectivity=kneighbors_graph(X, k, mode='distance', p=2).todense(), linkage='ward').fit_predict(X)),B=20, f=0.8))

plt.plot(n_neighbors_list, noisy_stability, label = "Noisy")
plt.plot(n_neighbors_list, noise_free_stability, label = "Noise-free")
plt.title("Stability of Agglomerative Clustering as a function of the number of neighbors")
plt.legend()


### Experiment on MNIST

TODO : develop this part (make the number of clusters vary, visualize the clusters (PCA ?))

In [ ]:
from sklearn.datasets import load_digits
mnist = load_digits()

mnist_stability = []
n_neighbors_list = np.arange(1,11,2)
for k in n_neighbors_list:
    mnist_stability.append(
        cluster_stability(mnist.data, 
            (lambda X, clusters: SpectralClustering(n_clusters=clusters, 
                            affinity="precomputed").fit_predict(kneighbors_graph(X, k, mode='distance', p=10).todense())),
            B=20, f=0.8))

plt.plot(n_neighbors_list, mnist_stability)
plt.title("Stability of Spectral Clustering as a function of the number of neighbors")
plt.legend()


## Semi-Supervised Learning

Choice of the dataset used : **Precise** which dataset you chose and why it is relevant for the semi-supervised learning Task

Advised datasets :

*Breast Cancer Wisconsin (Diagnostic) Database*

*MNIST binary even vs odd (multiple clusters inside each class)*

Feel free to use other datasets if they are relevant

In [ ]:
from sklearn.datasets import load_breast_cancer, load_digits
from sklearn.model_selection import train_test_split


### For all the next questions, use Cancer and Mnist classes to handle your data if you choose to use these one,
### You can also add more datasets but we advise you to handle them with this class for better readability
class semi_sup_dat:
    def __init__(self,data,p_unlabelled,name):
        # DON T CHANGE THE RANDOM STATES
        self.name = name
        if self.name == 'Mnist':
            # do an even vs odd binary classification :
            even = [0,2,4,6,8]
            Y = np.array([int(y in even) for y in data.target])
        else:
            Y = data.target
        X_lab, X_unlab, y_lab, y_unlab = train_test_split(data.data, Y, test_size=p_unlabelled, random_state=32)
        self.X_lab = X_lab
        self.X_unlab = X_unlab
        self.y_lab = y_lab
        self.y_unlab = y_unlab


# The following lines can be called later in the code to build a dataset with varying unlabelled proportion
p_unlabelled = 0.8 # You can change this value
cancer = load_breast_cancer()
Cancer = semi_sup_dat(cancer, p_unlabelled,'Cancer')

digits = load_digits()
Mnist = semi_sup_dat(digits, p_unlabelled,'Mnist')

### Question 9 : Complete the code in the box below

In [ ]:
#  Question 9  : Complete the function self_training
from sklearn.neighbors import KNeighborsClassifier

def self_training_kNN(X, neighbors = 3):
    X_lab = X.X_lab.copy()
    y_lab = X.y_lab.copy()    
    X_unlab = X.X_unlab.copy()
    y_unlab = X.y_unlab.copy() 
    
    for i in range(len(y_unlab)):
        kNN = KNeighborsClassifier(neighbors, n_jobs = -1)
        kNN.fit(X_lab,y_lab)
    
        y_tmp = kNN.predict_proba(X_unlab)
        
        #Get the most probable prediction
        #couple of indices: (index of sample, class)
        idx,c = np.unravel_index(np.argmax(y_tmp), y_tmp.shape)

        
        X_lab = np.vstack([X_lab,X_unlab[idx,:]])
        y_lab = np.hstack([y_lab, [c]])
        
        y_unlab[idx] = c
        X_unlab = np.delete(X_unlab,idx,0)
        
        
    return y_unlab


#When the 2 classes are 0,1, this is the error
def l1_loss (y_pred, data):
    return (np.linalg.norm(y_pred - data.y_unlab,1)/len(y_pred))

Let's test self-training on the cancer and mnist datasets

In [ ]:
y_pred = self_training_kNN(Cancer, 3)
print 100 * l1_loss(y_pred, Cancer)

y_pred_mnist = self_training_kNN(Mnist, 3)
print 100 * l1_loss(y_pred_mnist, Mnist)

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel # Or reimplement it yourself if your prefer
from scipy.linalg import block_diag

    

### Question 10 : Complete the code in the box below

We aim at solving 
$$\min_{f\in \mathcal{H}_\mathcal{K}} \frac{1}{l}\sum_{i = 1}^l (y_i - f(x_i))^2 + \lambda\Vert f \Vert_{\mathcal{H}_\mathcal{K}}^2 + \frac{\lambda_u}{u+l} f^\top L f$$


By applying the representer theorem, we get that there exits a vector $\alpha \in \mathbb{R}^{l+u}$ such that the solution is of the form $f(.) = \sum_{i=1}^{l+u} \mathcal{K}(.,x_i)\alpha_i$

Let us denote $K$ the kernel matrix $\lbrace\mathcal{K}(x_i,x_j)\rbrace_{i,j}$. Pluggin this expresssion into our minimization problem, we have that

$\begin{align}
\frac{1}{l}\sum_{i = 1}^l (y_i - f(x_i))^2 &= \frac{1}{l}\sum_{i = 1}^l (y_i - \sum_{j=1}^{l+u}\mathcal{K}(x_i,x_j)\alpha_j)^2\\
&= \frac{1}{l}\Vert y \Vert_2^2 - \frac{2}{l}\sum_{i=1}^{l}\sum_{j=1}^{l+u}y_i\alpha_j\mathcal{K}(x_i,x_j) + \frac{1}{l} \sum_{j=1}^{l+u}\sum_{l=1}^{l+u}\alpha_j\alpha_i\mathcal{K}(x_i,x_j)\\
&= \frac{1}{l}\Vert y \Vert_2^2 - \frac{2}{l} y^\top K \alpha + \frac{1}{l}\alpha^\top K \alpha
\end{align}$

where $y \in \mathbb{R}^{l+u}, \forall  j \geq l, y_i = 0$ is the zero-padded vector of known labels.


We also have that 

$\begin{align}
\Vert f \Vert_{\mathcal{H}_\mathcal{K}}^2 &=  <\sum_{i=1}^{l+u} \mathcal{K}(.,x_i)\alpha_i, \sum_{i=1}^{l+u} \mathcal{K}(.,x_j)\alpha_j >_{\mathcal{H}_\mathcal{K}}\\
&= \sum_{i,j} \alpha_i \alpha_j \mathcal{K}(x_i, x_j) \text{ by the reproducing property of }\mathcal{K(.,.)} \\
&= \alpha^\top K \alpha
\end{align}$

and 

$\begin{align}
\boldsymbol{f}^\top L \boldsymbol{f} &= \sum_{ij} f(x_i) l_{ij} f_(x_j)\\
&= \sum_{ij}\sum_{p,q}  \alpha_p K_{ip}l_{ij} K_{jq} \alpha_q\\
&= \alpha^\top K^\top L K\alpha
\end{align}$

Getting rid of the terms that do not play a role in the minimization, and taking the symmetry of $K$ into account, we can therefore rewrite our minimization problem as

$$\min_{\alpha \in \mathbb{R}^{l+u}}  \frac{1}{l}\alpha^\top K \alpha  - \frac{2}{l} y^\top K \alpha +  \lambda\alpha^\top K \alpha + \frac{\lambda_u}{u+l} \alpha^\top K L K\alpha$$

i.e.

$$\min_{\alpha \in \mathbb{R}^{l+u}}  \alpha^\top [(\frac{1}{l} + \lambda) K + \frac{\lambda_u}{u+l} K L K] \alpha  - \frac{2}{l} y^\top K \alpha$$


Since this is a convex minimization problem, at the optimum the gradient of the objective function vanishes, i.e. its solution $\alpha^\star$ verifies

$$[(\frac{1}{l} + \lambda) K + \frac{\lambda_u}{u+l} K L K]\alpha^\star = \frac{2}{l} K y$$

which finally yields 

$$ \alpha^\star = \frac{2}{l} [(\frac{1}{l} + \lambda) K + \frac{\lambda_u}{u+l} K L K]^{-1}K y$$

### Question 11 : Complete the code in the box below

### Question 12 : Add your answer here

-

-

-

### Question 13 : Complete the code in the box below

### Question 14 : Complete the code in the box below

##### Add your answer to the question here :

-

-

-

### Question 15 : Complete the code in the box below

### Question 16 : Complete the code in the box below

###### Describe your protocol here : 
-

-

-